In [37]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm


### Caricamento delle immagini in input in formato array numpy

In [49]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Distorto'
#DISTORSIONI = ['Speckle-m0-s', 'Speckle-m1-s', 'Speckle-m2-s', 'Artefatti Riflessione-']
DISTORSIONI = ['Speckle-m0-s', 'Speckle-m1-s', 'Speckle-m2-s']
GRADI_DISTORSIONE = 5

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [50]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    cartelle_soggetti = get_sotto_cartelle(path_cartella)
    for soggetto in tqdm(cartelle_soggetti):
        path_soggetto = f'{path_cartella}/{soggetto}'
        cartelle_acquisizioni = get_sotto_cartelle(path_soggetto)
        for acquisizione in cartelle_acquisizioni:
            path_acquisizione = f'{path_soggetto}/{acquisizione}'
            for filename in os.listdir(path_acquisizione):
                # if (filename.endswith(".jpg") or filename.endswith(".png"):
                if (filename.endswith("_1.jpg")):    
                    path_file = f'{path_acquisizione}/{filename}'
                    immagine = get_immagine_da_path(path_file)
                    immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [51]:
image_list = []
label_list = []

for distorsione in DISTORSIONI:
    path_distorsione = f'{IMAGE_PATH_FOLDER}/Dataset Sensori {distorsione}'
    for grado_distorsione in range(GRADI_DISTORSIONE):
        label = 100 - float(20 * (grado_distorsione))
        path_cartella_grado = f'{path_distorsione}{grado_distorsione}'
        immagini_grado = get_immagini_cartella(path_cartella_grado)
        image_list.extend(immagini_grado)
        label_to_append = [label for x in range(len(immagini_grado))] 
        label_list.extend(label_to_append)
        
        
        
        # for root, dirs, files in os.walk(path_cartella_grado):
        #     for file in files:
        #         if(file
        #        # print(root)
        #        # for _dir in dirs:
        #        #     print(_dir)
        #        for _file in files:
        #            print(_file)
        

100%|██████████| 14/14 [00:01<00:00, 10.73it/s]


In [52]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [53]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Un singolo output per il valore predetto
])

In [54]:
model.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [60]:
model.fit(x=train_images, y=train_weights, batch_size=1, epochs=30)

Epoch 1/30
3075/3075 [==============================] - 345s 112ms/step - loss: 3.4013 - mae: 3.4013
Epoch 2/30
3075/3075 [==============================] - 341s 111ms/step - loss: 3.2428 - mae: 3.2428
Epoch 3/30
3075/3075 [==============================] - 341s 111ms/step - loss: 3.1054 - mae: 3.1054
Epoch 4/30
3075/3075 [==============================] - 342s 111ms/step - loss: 3.1230 - mae: 3.1230
Epoch 5/30
3075/3075 [==============================] - 342s 111ms/step - loss: 3.1261 - mae: 3.1261
Epoch 6/30
3075/3075 [==============================] - 342s 111ms/step - loss: 2.9905 - mae: 2.9905
Epoch 7/30
3075/3075 [==============================] - 342s 111ms/step - loss: 3.0050 - mae: 3.0050
Epoch 8/30
3075/3075 [==============================] - 342s 111ms/step - loss: 2.8768 - mae: 2.8768
Epoch 9/30
3075/3075 [==============================] - 342s 111ms/step - loss: 2.8665 - mae: 2.8665
Epoch 10/30
3075/3075 [==============================] - 342s 111ms/step - loss: 2.9854 - m

In [56]:
predictions = model.predict(train_images)

2023-07-09 15:28:07.925549: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


97/97 [==============================] - 23s 218ms/step


In [57]:
predictions

array([[97.90513 ],
       [95.98184 ],
       [96.95886 ],
       ...,
       [24.281197],
       [22.211346],
       [22.889868]], dtype=float32)

In [ ]:
model.save('./Modelli/modello_pesi_inizializzati_v2')

INFO:tensorflow:Assets written to: ./Modelli/modello_pesi_inizializzati_v2/assets


INFO:tensorflow:Assets written to: ./Modelli/modello_pesi_inizializzati_v2/assets


In [63]:
max([i[0] for i in predictions]  - train_weights)

10.407417297363281